In [1]:
import requests
import pandas as pd
from configparser import ConfigParser

parser = ConfigParser()
parser.read('pipeline.conf')
api_key = parser.items('football-data')[0][1]
headers = {'X-AUTH-TOKEN':api_key}


In [2]:
# Chequeo si la api funciona
requests.get('https://api.football-data.org/v4/matches', headers=headers).status_code

200

In [3]:
url_teams = 'https://api.football-data.org/v4/competitions/PL/teams'
data_PL = requests.get(url=url_teams, headers=headers).json()
data_PL


{'count': 20,
 'filters': {'season': '2024'},
 'competition': {'id': 2021,
  'name': 'Premier League',
  'code': 'PL',
  'type': 'LEAGUE',
  'emblem': 'https://crests.football-data.org/PL.png'},
 'season': {'id': 2287,
  'startDate': '2024-08-16',
  'endDate': '2025-05-25',
  'currentMatchday': 1,
  'winner': None},
 'teams': [{'area': {'id': 2072,
    'name': 'England',
    'code': 'ENG',
    'flag': 'https://crests.football-data.org/770.svg'},
   'id': 57,
   'name': 'Arsenal FC',
   'shortName': 'Arsenal',
   'tla': 'ARS',
   'crest': 'https://crests.football-data.org/57.png',
   'address': '75 Drayton Park London N5 1BU',
   'website': 'http://www.arsenal.com',
   'founded': 1886,
   'clubColors': 'Red / White',
   'venue': 'Emirates Stadium',
   'runningCompetitions': [{'id': 2180,
     'name': 'Club Friendlies',
     'code': 'CLF',
     'type': 'PLAYOFFS',
     'emblem': None},
    {'id': 2021,
     'name': 'Premier League',
     'code': 'PL',
     'type': 'LEAGUE',
     'emblem'

In [4]:
teams_data = data_PL['teams']
teams_data

[{'area': {'id': 2072,
   'name': 'England',
   'code': 'ENG',
   'flag': 'https://crests.football-data.org/770.svg'},
  'id': 57,
  'name': 'Arsenal FC',
  'shortName': 'Arsenal',
  'tla': 'ARS',
  'crest': 'https://crests.football-data.org/57.png',
  'address': '75 Drayton Park London N5 1BU',
  'website': 'http://www.arsenal.com',
  'founded': 1886,
  'clubColors': 'Red / White',
  'venue': 'Emirates Stadium',
  'runningCompetitions': [{'id': 2180,
    'name': 'Club Friendlies',
    'code': 'CLF',
    'type': 'PLAYOFFS',
    'emblem': None},
   {'id': 2021,
    'name': 'Premier League',
    'code': 'PL',
    'type': 'LEAGUE',
    'emblem': 'https://crests.football-data.org/PL.png'}],
  'coach': {'id': 179744,
   'firstName': '',
   'lastName': 'Mikel Arteta',
   'name': 'Mikel Arteta',
   'dateOfBirth': '1982-03-26',
   'nationality': 'Spain',
   'contract': {'start': '2019-12', 'until': '2025-06'}},
  'squad': [{'id': 4832,
    'name': 'David Raya',
    'position': 'Goalkeeper',
  

In [5]:
teams = []
for team in teams_data:
    coach = f'{team['coach']['firstName']} {team['coach']['lastName']}'
    teams.append(dict(id=team['id'],team=team['name'],Stadium=team['venue'],dt=coach,lastUpdated=team['lastUpdated']))

In [6]:
columns = ['id','team','Stadium','dt','lastUpdated']
df_2024_2025_PL = pd.DataFrame(data=teams,columns=columns)
# id , team , Stadium , dt , LastUpdated
df_2024_2025_PL

,id,team,Stadium,dt,lastUpdated
0,57,Arsenal FC,Emirates Stadium,Mikel Arteta,2022-02-10T19:48:56Z
1,58,Aston Villa FC,Villa Park,Unai Emery,2022-04-03T16:22:14Z
2,61,Chelsea FC,Stamford Bridge,Enzo Maresca,2022-02-10T19:24:40Z
3,62,Everton FC,Goodison Park,Sean Dyche,2022-02-10T19:47:42Z
4,63,Fulham FC,Craven Cottage,Marco Silva,2024-07-29T17:16:11Z
5,64,Liverpool FC,Anfield,Arne Slot,2022-02-10T19:30:22Z
6,65,Manchester City FC,Etihad Stadium,Pep Guardiola,2022-02-10T19:48:37Z
7,66,Manchester United FC,Old Trafford,Erik ten Hag,2022-02-10T19:27:46Z
8,67,Newcastle United FC,St. James' Park,Eddie Howe,2022-09-28T18:51:07Z
9,73,Tottenham Hotspur FC,Tottenham Hotspur Stadium,Ange Postecoglou,2024-05-11T21:39:03Z


In [7]:
squads = []
for team in teams_data:
    squad_data = team['squad']
    squad = []
    for player_data in squad_data:
        player = dict(id=player_data['id'],name=player_data['name'],position=player_data['position'],nationality=player_data['nationality']) 
        squad.append(player)
    
    squads.append(dict(id_team=team['id'],squad=squad))
    
squads

[{'id_team': 57,
  'squad': [{'id': 4832,
    'name': 'David Raya',
    'position': 'Goalkeeper',
    'nationality': 'Spain'},
   {'id': 5530,
    'name': 'Aaron Ramsdale',
    'position': 'Goalkeeper',
    'nationality': 'England'},
   {'id': 153843,
    'name': 'Karl Jakob Hein',
    'position': 'Goalkeeper',
    'nationality': 'Estonia'},
   {'id': 147286,
    'name': 'Jakub Kiwior',
    'position': 'Left-Back',
    'nationality': 'Poland'},
   {'id': 133512,
    'name': 'Riccardo Calafiori',
    'position': 'Centre-Back',
    'nationality': 'Italy'},
   {'id': 112948,
    'name': 'Fabio Vieira',
    'position': 'Attacking Midfield',
    'nationality': 'Portugal'},
   {'id': 99813,
    'name': 'Bukayo Saka',
    'position': 'Right Winger',
    'nationality': 'England'},
   {'id': 98816,
    'name': 'Jurrien Timber',
    'position': 'Centre-Back',
    'nationality': 'Netherlands'},
   {'id': 80171,
    'name': 'William Saliba',
    'position': 'Centre-Back',
    'nationality': 'Franc

In [8]:
df_squads = pd.DataFrame(data=squads)
df_squads
# Cada registro muestra el id_team y una lista de jugadores


,id_team,squad
0,57,"[{'id': 4832, 'name': 'David Raya', 'position'..."
1,58,"[{'id': 3141, 'name': 'Emiliano Martínez', 'po..."
2,61,"[{'id': 3189, 'name': 'Kepa Arrizabalaga', 'po..."
3,62,"[{'id': 3309, 'name': 'Jordan Pickford', 'posi..."
4,63,"[{'id': 3174, 'name': 'Bernd Leno', 'position'..."
5,64,"[{'id': 1795, 'name': 'Alisson', 'position': '..."
6,65,"[{'id': 3222, 'name': 'Ederson', 'position': '..."
7,66,"[{'id': 7544, 'name': 'André Onana', 'position..."
8,67,"[{'id': 3310, 'name': 'Nick Pope', 'position':..."
9,73,"[{'id': 3086, 'name': 'Guglielmo Vicario', 'po..."


In [9]:
# Normalizamos el json para que cada jugador sea un registro con su respectivo id_team
df_squads = pd.json_normalize(data=squads,record_path='squad',meta='id_team')
df_squads

,id,name,position,nationality,id_team
0,4832,David Raya,Goalkeeper,Spain,57
1,5530,Aaron Ramsdale,Goalkeeper,England,57
2,153843,Karl Jakob Hein,Goalkeeper,Estonia,57
3,147286,Jakub Kiwior,Left-Back,Poland,57
4,133512,Riccardo Calafiori,Centre-Back,Italy,57
...,...,...,...,...,...
592,4427,Chris Mepham,Centre-Back,Wales,1044
593,4417,Antoine Semenyo,Right Winger,Ghana,1044
594,4321,David Brooks,Right Winger,Wales,1044
595,3327,Lewis Cook,Central Midfield,England,1044


In [15]:
data_matches = requests.get('https://api.football-data.org/v4/competitions/PL/matches', headers=headers).json()

In [26]:
# Estado de todos los partidos matchday, id, status, utcDate
for match in data_matches['matches']:
    print(match['matchday'],match['id'],match['status'],match['utcDate'])

match['utcDate']
# Cuando cada partido este en estado finished, se cargaran datos a la base de datos, y tambien tenemos
# la utcdate para hacer la carga incremental
# Cada vez que se ejecute el script, se cargaran los datos de los partidos que esten en estado finished y que no se hayan cargado previamente  
# El dato que determinara cual fue la ultima carga estara en un .json que se actualizara cada vez que se ejecute el script  

1 497410 TIMED 2024-08-16T19:00:00Z
1 497411 TIMED 2024-08-17T11:30:00Z
1 497412 TIMED 2024-08-17T14:00:00Z
1 497413 TIMED 2024-08-17T14:00:00Z
1 497414 TIMED 2024-08-17T14:00:00Z
1 497415 TIMED 2024-08-17T14:00:00Z
1 497416 TIMED 2024-08-17T16:30:00Z
1 497417 TIMED 2024-08-18T13:00:00Z
1 497418 TIMED 2024-08-18T15:30:00Z
1 497419 TIMED 2024-08-19T19:00:00Z
2 497422 TIMED 2024-08-24T11:30:00Z
2 497423 TIMED 2024-08-24T14:00:00Z
2 497424 TIMED 2024-08-24T14:00:00Z
2 497426 TIMED 2024-08-24T14:00:00Z
2 497427 TIMED 2024-08-24T14:00:00Z
2 497428 TIMED 2024-08-24T14:00:00Z
2 497421 TIMED 2024-08-24T16:30:00Z
2 497420 TIMED 2024-08-25T13:00:00Z
2 497429 TIMED 2024-08-25T13:00:00Z
2 497425 TIMED 2024-08-25T15:30:00Z
3 497430 TIMED 2024-08-31T11:30:00Z
3 497431 TIMED 2024-08-31T14:00:00Z
3 497433 TIMED 2024-08-31T14:00:00Z
3 497434 TIMED 2024-08-31T14:00:00Z
3 497435 TIMED 2024-08-31T14:00:00Z
3 497438 TIMED 2024-08-31T14:00:00Z
3 497439 TIMED 2024-08-31T16:30:00Z
3 497432 TIMED 2024-09-01T14

'2025-05-25T15:00:00Z'